###This uses the official nnunet implementation

Note: the implementation includes 5 fold cross validation built in -- throw everything into the training set

In [1]:
%pip install nnunetv2

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pytest-shutil

                                              0.0/320.9 kB ? eta -:--:--
     ---------------------------            235.5/320.9 kB 7.3 MB/s eta 0:00:01
     -------------------------------------- 320.9/320.9 kB 6.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
torch.__version__

'2.1.0.dev20230602+cu121'

In [6]:
#grabbed from DatasetStatistics.ipynb

import glob
import SimpleITK as sitk

def hasMask(name):
    files = glob.glob(name+"/*.mha")

    reader = sitk.ImageFileReader()

    if(len(files)<2):
        return False
    return True


def axial(name):
    files = glob.glob(name+"/*.mha")
    if("ax" in files[0] and "ax" in files[1]):
        return True
    return False



CCF_count = 0

CCF_missing = [] #missing masks
CCF_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled/*'):
    
    if not (hasMask(name)):
        CCF_missing.append(name)
        continue
    if axial(name):
        CCF_count +=1
        CCF_list.append(name)
    

print("Total CCF: " + str(CCF_count))
print("Missing masks for CCF:" + str(CCF_missing))




UH_count = 0

UH_missing = [] #missing masks
UH_list = []

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled/*'):
    

    if not (hasMask(name)):
        UH_missing.append(name)
        continue
    if axial(name):
        UH_count +=1
        UH_list.append(name)

    

print("Total UH: " + str(UH_count))
print("Missing masks for UH:" + str(UH_missing))

print("Total: " +str(UH_count+CCF_count))
print("Expected: 166")



com_List = UH_list + CCF_list


Total CCF: 66
Missing masks for CCF:[]
Total UH: 100
Missing masks for UH:[]
Total: 166
Expected: 166


In [37]:
import pandas as pd
import numpy as np 

def removeNaN(x):
    temp = x[~pd.isnull(x)]
    return temp

split = pd.read_excel(r'G:\Shared drives\INVent\mfk58\New-Dataset-split.xlsx',sheet_name='splits')
split = split.to_numpy()

trset = removeNaN(split[:,0:3].flatten()) #first three columns
ext = removeNaN(split[:,3].flatten())



In [41]:
counts = [0,0,0]

import shutil



def copyToDirs(source: str, holdout: bool):
    if not holdout:
        images = r"G:\Shared drives\INVent\mfk58\trainingSet"
        labels = r"G:\Shared drives\INVent\mfk58\trainingLabels"
    else:
        images=r"G:\Shared drives\INVent\mfk58\holdoutSet"
        labels = r"G:\Shared drives\INVent\mfk58\holdoutLabels"
    
    files = glob.glob(dir+"/*.mha")
    label = ""
    img = ""
    if "label" in files[0]:
        label = files[0]
        img = files[1]
    else:
        label = files[1]
        img = files[0]
    
    shutil.copy2(label, labels)
    shutil.copy2(img, images)



for dir in com_List:
    files = glob.glob(dir+"/*.mha")
    label = ""
    img = ""
    if "label" in files[0]:
        label = files[0]
        img = files[1]
    else:
        label = files[1]
        img = files[0]
    
    s = img.replace(dir+"\\","") #should hopefully remove the portion that is not included in the actual dataset splits value
    s = s.replace(".mha","")

    if s in trset:
        counts[0] += 1
        copyToDirs(dir, False)
    elif s in ext:
        counts[1] += 1
        copyToDirs(dir, True)
    else:
        counts[2] += 1



In [56]:
import glob
images = r"G:\Shared drives\INVent\mfk58\trainingSet"
labels = r"G:\Shared drives\INVent\mfk58\trainingLabels"


e_images=r"G:\Shared drives\INVent\mfk58\holdoutSet"
e_labels = r"G:\Shared drives\INVent\mfk58\holdoutLabels"




In [68]:
import SimpleITK as sitk
def filterImage(img: str, val):
    r = sitk.ImageFileReader()
    r.SetFileName(img)
    a = r.Execute()
    arr = sitk.GetArrayFromImage(a)
    arr[arr != val] = 0
    changed = sitk.GetImageFromArray(arr)

    w = sitk.ImageFileWriter()
    w.SetFileName(img)
    changed.CopyInformation(a)
    w.Execute(changed)

e_labels = r"G:\Shared drives\INVent\mfk58\holdoutLabels"
labels = r"G:\Shared drives\INVent\mfk58\trainingLabels"
for file in glob.glob(e_labels+"/*.mha"):
    filterImage(file,1)
for file in glob.glob(labels+"/*.mha"):
    filterImage(file,1)

In [1]:
import glob
import SimpleITK as sitk
import numpy as np
from scipy import ndimage
import pandas as pd 

def center(arr):
    out = []
    arr[arr!=-1] = 1
    for dim in ndimage.center_of_mass(arr):
        out.append(round(dim))
    return out

def get_bounding_box(x):
    """ Calculates the bounding box of a ndarray"""
    mask = x == 0
    bbox = []
    all_axis = np.arange(x.ndim)
    for kdim in all_axis:
        nk_dim = np.delete(all_axis, kdim)
        mask_i = mask.all(axis=tuple(nk_dim))
        dmask_i = np.diff(mask_i)
        idx_i = np.nonzero(dmask_i)[0]

        low = min( idx_i )
        high = max( idx_i )
        s = slice(low + 1, high + 1)
        bbox.append(s)
    return bbox

def findCropBox(cen:tuple, img: tuple, crop:tuple):
    box = []
    for k in range(0,len(crop)):
        exp = round((crop[k]-1)/2) # the distance in any one direction, again assumes odd size
        if ((exp+cen[k]) < img[k]) and (cen[k] - exp >= 0): #both directions within the inner bound -- perfect, move on
            box.append(slice(cen[k] - exp,exp+cen[k]+1)) #center the slice on the center
        else:
            #THE CROP TO SIZE SHOULD NOT EXCEED THE IMG SIZE, SO THIS SHOULDN'T BE AN ISSUE IN BOTH DIRECTIONS
            if not ((exp+cen[k]) < img[k]): #the outer boundary is expanded past
                box.append(slice(img[k]-crop[k],img[k])) #push directly against the edge of the image
            else: # went below 0
                box.append(slice(0,crop[k]))
    if(len(crop)<3):
        box.append(slice(0,img[2]))
    return tuple(box)

def Reverse(tuples): ##needed because the simple ITK getarrayfromimage gives as z,y,x  - I am currently treating as x,y,z beacuse sitk images are xyz listed
    new_tup = tuples[::-1]
    return new_tup


#assumes that all of the dimensions in size are ODD
#should work for cropping in 2D (i.e., ignoring depth) and with 3D
def cropAll(size: tuple, img_dir: str, lbl_dir: str, xlsxfile: str, s_name:str="output"):
    imgs = glob.glob(img_dir+"\\*.mha")
    imgs.sort()
    masks = glob.glob(lbl_dir+"\\*.mha")
    masks.sort()
    assert len(imgs) == len(masks)

    r = sitk.ImageFileReader()
    w = sitk.ImageFileWriter()

    toExcel = [[],[]]

    for i in range(0,len(masks)):
        print(masks[i])
        r.SetFileName(masks[i])
        img = r.Execute()
        a = sitk.GetArrayFromImage(img)
        bbx = get_bounding_box(a)
        
        arr = a[bbx[0].start:bbx[0].stop,bbx[1].start:bbx[1].stop,bbx[2].start:bbx[2].stop]
        c = center(arr)
        for j in range(0,len(c)):
            c[j] += bbx[j].start
        print(c)
        print(a.shape)
        cbox = findCropBox(Reverse(tuple(c)),Reverse(a.shape),size)
        cropped = img[cbox[0],cbox[1],cbox[2]]
        w.SetFileName(masks[i])
        w.Execute(cropped)


        w.SetFileName(imgs[i])
        r.SetFileName(imgs[i])
        w.Execute(r.Execute()[cbox[0],cbox[1],cbox[2]])

        toExcel[0].append(masks[i])
        toExcel[1].append(str(cbox))
    writer = pd.ExcelWriter(xlsxfile, engine='xlsxwriter')
    df = pd.DataFrame(toExcel).T
    df.to_excel(writer,sheet_name=s_name,index=False)
    writer.close()












In [146]:
r = sitk.ImageFileReader()
r.SetFileName(r"G:\Shared drives\INVent\mfk58\trainingLabels\UH_RectalCA_168_pre_ax_raw_label_resampled.mha")
img = sitk.GetArrayFromImage(r.Execute())
bbx = get_bounding_box(img)
print(bbx)
arr = img[bbx[0].start:bbx[0].stop,bbx[1].start:bbx[1].stop,bbx[2].start:bbx[2].stop]
print(center(arr)) 

[slice(35, 107, None), slice(74, 179, None), slice(61, 134, None)]
[36, 52, 36]


In [17]:
cropAll(tuple([145,135]),r"G:\Shared drives\INVent\mfk58\holdoutSet",r"G:\Shared drives\INVent\mfk58\holdoutLabels", r'G:\Shared drives\INVent\mfk58\holdoutcroplocations.xlsx')
cropAll(tuple([145,135]),r"G:\Shared drives\INVent\mfk58\trainingSet",r"G:\Shared drives\INVent\mfk58\trainingLabels", r'G:\Shared drives\INVent\mfk58\trainingcroplocations.xlsx')

G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_014_pre_ax_label_raw_resampled.mha
[99, 152, 104]
(198, 230, 230)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_017_pre_ax_label_raw_resampled.mha
[57, 150, 111]
(84, 230, 230)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_031_pre_ax_label_raw_resampled.mha
[96, 179, 152]
(216, 300, 300)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_041_pre_ax_label_raw_resampled.mha
[162, 138, 140]
(279, 227, 290)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_053_pre_ax_label_raw_resampled.mha
[132, 131, 112]
(187, 200, 200)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_076_pre_ax_label_raw_resampled.mha
[129, 98, 93]
(180, 160, 160)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_089_pre_ax_label_raw_resampled.mha
[117, 132, 102]
(286, 200, 200)
G:\Shared drives\INVent\mfk58\holdoutLabels\CCF_RectalCA_135_pre_ax_label_raw_resampled.mha
[77, 120, 102]
(168, 202, 202)
G:\Shared drive

In [18]:
import glob
import SimpleITK as sitk

import os

def renameFile(file: str, dir: str, label: bool):

    c = file
    s = ""

    if "CCF" in file:
        s = "CCF"
        c = c.replace("CCF_RectalCA","")
    elif "UH" in file:
        s = "UH"
        c = c.replace("UH_RectalCA","")
    else:
        raise Exception("Did not meet criteria for renaming")
    
    s = s + c[:4]

    
    if not label:
        s  = s + "_0000.mha"
    else:
        s = s + ".mha"
   
    os.rename(dir+"/"+file,dir+"/"+s)




paths = [r"G:\Shared drives\INVent\mfk58\trainingSet", r"G:\Shared drives\INVent\mfk58\trainingLabels", r"G:\Shared drives\INVent\mfk58\holdoutSet", r"G:\Shared drives\INVent\mfk58\holdoutLabels"]
r = sitk.ImageFileReader()
for path in paths:
    isLabel = False
    if "Labels" in path:
        isLabel = True
    files = glob.glob(path + "/*.mha")
    for file in files:
        print(file)
        r.SetFileName(file)
        r.LoadPrivateTagsOn()
        r.ReadImageInformation()

        xsz, ysz, zsz = r.GetSize()
        assert xsz == 145
        assert ysz == 135
        
        renameFile(file.replace(path+"\\",""),path,isLabel)
        
    

G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_004_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_012_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_009_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_013_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_011_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_002_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_008_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_006_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_003_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_033_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_027_pre_ax_raw_resampled.mha
G:\Shared drives\INVent\mfk58\trainingSet\UH_RectalCA_026_pre_ax_raw_resampled.mha
G:\S

In [4]:
import json
import os
def save_json(obj, file: str, indent: int = 4, sort_keys: bool = True) -> None:
    with open(file, 'w') as f:
        json.dump(obj, f, sort_keys=sort_keys, indent=indent)

def generate_dataset_json(output_folder: str,
                          channel_names: dict,
                          labels: dict,
                          num_training_cases: int,
                          file_ending: str,
                          regions_class_order= None,
                          dataset_name: str = None, reference: str = None, release: str = None, license: str = None,
                          description: str = None,
                          overwrite_image_reader_writer: str = None, **kwargs):
    """
    Generates a dataset.json file in the output folder

    channel_names:
        Channel names must map the index to the name of the channel, example:
        {
            0: 'T1',
            1: 'CT'
        }
        Note that the channel names may influence the normalization scheme!! Learn more in the documentation.

    labels:
        This will tell nnU-Net what labels to expect. Important: This will also determine whether you use region-based training or not.
        Example regular labels:
        {
            'background': 0,
            'left atrium': 1,
            'some other label': 2
        }
        Example region-based training:
        {
            'background': 0,
            'whole tumor': (1, 2, 3),
            'tumor core': (2, 3),
            'enhancing tumor': 3
        }

        Remember that nnU-Net expects consecutive values for labels! nnU-Net also expects 0 to be background!

    num_training_cases: is used to double check all cases are there!

    file_ending: needed for finding the files correctly. IMPORTANT! File endings must match between images and
    segmentations!

    dataset_name, reference, release, license, description: self-explanatory and not used by nnU-Net. Just for
    completeness and as a reminder that these would be great!

    overwrite_image_reader_writer: If you need a special IO class for your dataset you can derive it from
    BaseReaderWriter, place it into nnunet.imageio and reference it here by name

    kwargs: whatever you put here will be placed in the dataset.json as well

    """
    has_regions: bool = any([isinstance(i, (tuple, list)) and len(i) > 1 for i in labels.values()])
    if has_regions:
        assert regions_class_order is not None, f"You have defined regions but regions_class_order is not set. " \
                                                f"You need that."
    # channel names need strings as keys
    keys = list(channel_names.keys())
    for k in keys:
        if not isinstance(k, str):
            channel_names[str(k)] = channel_names[k]
            del channel_names[k]

    # labels need ints as values
    for l in labels.keys():
        value = labels[l]
        if isinstance(value, (tuple, list)):
            value = tuple([int(i) for i in value])
            labels[l] = value
        else:
            labels[l] = int(labels[l])

    dataset_json = {
        'channel_names': channel_names,  # previously this was called 'modality'. I didnt like this so this is
        # channel_names now. Live with it.
        'labels': labels,
        'numTraining': num_training_cases,
        'file_ending': file_ending,
    }

    if dataset_name is not None:
        dataset_json['name'] = dataset_name
    if reference is not None:
        dataset_json['reference'] = reference
    if release is not None:
        dataset_json['release'] = release
    if license is not None:
        dataset_json['licence'] = license
    if description is not None:
        dataset_json['description'] = description
    if overwrite_image_reader_writer is not None:
        dataset_json['overwrite_image_reader_writer'] = overwrite_image_reader_writer
    if regions_class_order is not None:
        dataset_json['regions_class_order'] = regions_class_order

    dataset_json.update(kwargs)

    save_json(dataset_json, os.path.join(output_folder, 'dataset.json'), sort_keys=False)




generate_dataset_json(r"G:\Shared drives\INVent\mfk58\nnUNet_raw\Dataset167_RectalTumor",{0:'T2'},{'background':0,'tumor':1},127,".mha",dataset_name="RectalTumor",overwrite_image_reader_writer="SimpleITKIO")

### Things to do with HPC
Changed: see hpc_files!



## Next:

Running coronal scans haha

In [3]:
import glob
def coronal(name):
    files = glob.glob(name+"/*.mha")
    return ("cor" in files[0] and "cor" in files[1])

def sagittal(name):
    files = glob.glob(name+"/*.mha")
    return ("sag" in files[0] and "sag" in files[1])


sagittalScans = []
UHsag = 0
CCFsag = 0 

coronalScans = []
UHcor = 0
CCFcor = 0

for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/UH_Pre_Resampled/*'):
    

    if coronal(name):
        UHcor +=1
        coronalScans.append(name)
    if sagittal(name):
        UHsag += 1
        sagittalScans.append(name)



for name in glob.glob('G:/Shared drives/INVent/mfk58/Dataset/CCF_Pre_Resampled/*'):
    

    if coronal(name):
        CCFcor +=1
        coronalScans.append(name)
    if sagittal(name):
        CCFsag += 1
        sagittalScans.append(name)

print("Coronal Scans: " + str(len(coronalScans)))
print("from UH "+str(UHcor))
print("from CCF: "+str(CCFcor))

print("sagittal scans: "+str(len(sagittalScans)))
print("from UH "+ str(UHsag))
print("from CCF "+str(CCFsag))

Coronal Scans: 29
from UH 21
from CCF: 8
sagittal scans: 0
from UH 0
from CCF 0


In [4]:
coronalScans.sort()

In [8]:
for i in range(0, len(coronalScans)):
    c = coronalScans[i]
    s = ""
    if "CCF" in c:
        s = "CCF "
    elif "UH" in c:
        s = "UH "
    s += c.split("\\")[-1]
    print(s)
    coronalScans[i] = s

CCF RectalCA_034
CCF RectalCA_035
CCF RectalCA_043
CCF RectalCA_045
CCF RectalCA_091
CCF RectalCA_119
CCF RectalCA_120
CCF RectalCA_136
UH RectalCA_001
UH RectalCA_005
UH RectalCA_007
UH RectalCA_010
UH RectalCA_017
UH RectalCA_019
UH RectalCA_025
UH RectalCA_029
UH RectalCA_039
UH RectalCA_043
UH RectalCA_054
UH RectalCA_057
UH RectalCA_063
UH RectalCA_067
UH RectalCA_069
UH RectalCA_071
UH RectalCA_074
UH RectalCA_075
UH RectalCA_086
UH RectalCA_089
UH RectalCA_137


In [5]:
copy_to = r"G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs"
import shutil
import glob
for dir in coronalScans:
    imgs = glob.glob(dir+"/*.mha")
    file = imgs[1] if "label" in imgs[0] else imgs[0]
    shutil.copy2(file,copy_to)

In [6]:
import glob
import SimpleITK as sitk

import os

def renameFile(file: str, dir: str, label: bool):

    c = file
    s = ""

    if "CCF" in file:
        s = "CCF"
        c = c.replace("CCF_RectalCA","")
    elif "UH" in file:
        s = "UH"
        c = c.replace("UH_RectalCA","")
    else:
        raise Exception("Did not meet criteria for renaming")
    
    s = s + c[:4]

    
    if not label:
        s  = s + "_0000.mha"
    else:
        s = s + ".mha"
   
    os.rename(dir+"/"+file,dir+"/"+s)




paths = [r"G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs"]
r = sitk.ImageFileReader()
for path in paths:
    isLabel = False
    if "Labels" in path:
        isLabel = True
    files = glob.glob(path + "/*.mha")
    for file in files:
        print(file)
        r.SetFileName(file)
        r.LoadPrivateTagsOn()
        r.ReadImageInformation()

        
        renameFile(file.replace(path+"\\",""),path,isLabel)
        

G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_034_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_035_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_043_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_045_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_091_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_119_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_120_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\CCF_RectalCA_136_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\UH_RectalCA_001_pre_cor_raw_resampled.mha
G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs\UH_RectalCA_005_pre_cor_raw_resamp

In [3]:
import SimpleITK as sitk
def filterImage(img: str, val):
    r = sitk.ImageFileReader()
    r.SetFileName(img)
    a = r.Execute()
    arr = sitk.GetArrayFromImage(a)
    arr[arr != val] = 0
    changed = sitk.GetImageFromArray(arr)

    w = sitk.ImageFileWriter()
    w.SetFileName(img)
    changed.CopyInformation(a)
    w.Execute(changed)

import glob
for file in glob.glob(r"G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\*.mha"):
    filterImage(file,1)

In [2]:
cropAll(tuple([145,135]),r"G:\Shared drives\INVent\mfk58\ImageProcessing\CoronalImgs",r"G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels", r'G:\Shared drives\INVent\mfk58\coronalcroplocations.xlsx')

G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_034.mha
[51, 92, 82]
(120, 230, 173)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_035.mha
[27, 49, 121]
(120, 230, 230)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_043.mha
[81, 48, 99]
(156, 200, 200)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_045.mha
[46, 69, 115]
(180, 240, 240)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_091.mha
[86, 118, 93]
(144, 240, 191)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_119.mha
[75, 135, 175]
(144, 350, 350)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_120.mha
[33, 91, 109]
(72, 240, 240)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\CCF_136.mha
[47, 87, 130]
(101, 250, 250)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\UH_001.mha
[26, 69, 95]
(108, 200, 200)
G:\Shared drives\INVent\mfk58\ImageProcessing\coronalLabels\UH_005.mha
[55, 81, 95]
(1